# ⚛️ LiH — Ground-State VQE vs Bond Length (Noiseless, UCC-D)

This notebook performs a **bond-length scan** for **lithium hydride (LiH)** using
the production VQE API:

```python
from vqe.core import run_vqe_geometry_scan
```

We study how the **ground-state energy** varies as a function of the Li–H bond
distance using a **UCC doubles (UCC-D)** ansatz.

---

## Simulation setup

- Molecule: **LiH**
- Geometry: Li–H bond length scan
- Distances: **1.1 Å → 2.1 Å** (10 points)
- Basis: **STO-3G**
- Ansatz: **UCC-D** (doubles only)
- Optimizer: **GradientDescent**
- Device: `default.qubit`
- Noise: **None** (noiseless)

All heavy lifting (Hamiltonian construction, ansatz building, optimisation loop,
caching, and plotting) is handled by the package.

In [ ]:
import numpy as np
import sys
import os

# Allow local import when running from notebooks/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.core import run_vqe_geometry_scan

# 🧬 Bond-Length Grid

We define a simple one-dimensional scan over the Li–H bond length.

In [ ]:
# Li–H bond lengths in Ångström
bond_lengths = np.linspace(1.1, 2.1, 10)
print("Bond-length grid (Å):", bond_lengths)

# 🚀 Run Geometry Scan via `run_vqe_geometry_scan`

`run_vqe_geometry_scan`:

- Uses the internal LiH bond-length geometry generator
- Calls `run_vqe` under the hood for each bond length
- Reuses cached runs when available
- Produces a **Ground-state energy vs bond length** plot and saves it to disk

We use:
- Ansatz: **UCC-D** (doubles only)
- Optimizer: **GradientDescent**
- Single seed: `0`
- 25 optimisation steps per geometry

In [ ]:
results = run_vqe_geometry_scan(
    molecule="LiH_BOND",         # uses the LiH bond-length geometry helper
    param_name="bond",
    param_values=bond_lengths,
    ansatz_name="UCC-D",
    optimizer_name="GradientDescent",
    steps=25,
    stepsize=0.8,
    seeds=[0],
    force=False,
    mapping="jordan_wigner",
    show=True,                   # display the package-generated plot
)

# 📌 Extract Approximate Equilibrium Bond Length

`run_vqe_geometry_scan` returns a list of tuples:

\[
(r, \langle E \rangle, \sigma_E)
\]

where:
- \( r \) is the bond length
- \( \langle E \rangle \) is the mean ground-state energy over seeds
- \( \sigma_E \) is the standard deviation

In [ ]:
# Unpack results into arrays
params, mean_E, std_E = zip(*results)
params = np.array(params, dtype=float)
mean_E = np.array(mean_E, dtype=float)
std_E = np.array(std_E, dtype=float)

# Find approximate minimum
min_idx = int(np.argmin(mean_E))
r_opt = params[min_idx]
E_opt = mean_E[min_idx]
dE_opt = std_E[min_idx]

print(f"\nEstimated equilibrium bond length (Li–H): {r_opt:.3f} Å")
print(f"Minimum VQE ground-state energy: {E_opt:.8f} ± {dE_opt:.8f} Ha")

---
## Summary

In this notebook you:

- Used the **production VQE API** (`run_vqe_geometry_scan`) as a pure client
- Performed a **noiseless UCC-D VQE** bond-length scan for **LiH**
- Obtained a **Ground-state energy vs bond length** curve
- Extracted an approximate **equilibrium bond length** from the VQE data

All ansatz construction, Hamiltonian generation, device management,
optimisation, caching, and plotting were handled by the package.